### Importing Important Libraries

Steps To Be Followed

Importing necessary Libraries
Creating S3 bucket
Mapping train And Test Data in S3
Mapping The path of the models in S3

In [ ]:
##https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/multi_model_xgboost_home_value/xgboost_multi_model_endpoint_home_value.ipynb

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import image_uris
from sagemaker.session import Session

In [2]:
bucket_name = "bankzabhi"
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

ap-south-1


In [3]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'ap-south-1':
        s3.create_bucket(Bucket=bucket_name,CreateBucketConfiguration={'LocationConstraint':'ap-south-1'})
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [4]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankzabhi/xgboost-as-a-built-in-algo/output


Downloading The Dataset And Storing in S3

In [5]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [6]:
model_data

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,1,999,0,1,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
41184,46,1,999,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0
41185,56,2,999,0,1,1,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
41186,44,1,999,0,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1


In [7]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [8]:
### Saving Train And Test Into Buckets
## We start with Train Data
## in aws sagemaker first column should be independent column
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

In [9]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [10]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

Building Models Xgboot- Inbuilt Algorithm

In [11]:
# this line automatically looks for the XGBoost image URI which are stored in amazon cotainers and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = image_uris.retrieve(region=boto3.Session().region_name,
                          framework='xgboost', 
                          version='1')

In [12]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [13]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', 
                                          volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=200,
                                          max_wait=400)

In [14]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

2021-05-16 20:40:20 Starting - Starting the training job...
2021-05-16 20:40:21 Starting - Launching requested ML instancesProfilerReport-1621197619: InProgress
......
2021-05-16 20:41:34 Starting - Preparing the instances for training......
2021-05-16 20:42:41 Downloading - Downloading input data
2021-05-16 20:42:41 Training - Downloading the training image..Arguments: train
[2021-05-16:20:42:55:INFO] Running standalone xgboost training.
[2021-05-16:20:42:55:INFO] File size need to be processed in the node: 4.83mb. Available memory size in the node: 23817.03mb
[2021-05-16:20:42:55:INFO] Determined delimiter of CSV input is ','
[20:42:55] S3DistributionType set as FullyReplicated
[20:42:56] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-05-16:20:42:56:INFO] Determined delimiter of CSV input is ','
[20:42:56] S3DistributionType set as FullyReplicated
[20:42:56] 12357x59 matrix with 729063 entries loaded from /opt/ml/

Deploy Machine Learning Model As Endpoints

In [15]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

-------------!

Prediction of the Test Data

In [19]:
from sagemaker.predictor import CSVSerializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [20]:
predictions_array

array([0.05578489, 0.04585116, 0.05088526, ..., 0.03450042, 0.03321051,
       0.03635123])

In [21]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10784)    36% (152)
Purchase        10% (1149)     64% (272) 



In [22]:
#Deleting The Endpoints

sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


[{'ResponseMetadata': {'RequestId': 'XQ36G9AKD7ZP57YS',
   'HostId': '3xO2aUXQrQmXWKP7iCJ+T6UjC5+8fTD1MhN7j7cG+pkM6AtARHAOZbVWyVF0uC8XtU+g/IEzca4=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '3xO2aUXQrQmXWKP7iCJ+T6UjC5+8fTD1MhN7j7cG+pkM6AtARHAOZbVWyVF0uC8XtU+g/IEzca4=',
    'x-amz-request-id': 'XQ36G9AKD7ZP57YS',
    'date': 'Sun, 16 May 2021 20:54:16 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2021-05-16-20-40-19-843/rule-output/ProfilerReport-1621197619/profiler-output/profiler-reports/GPUMemoryIncrease.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2021-05-16-20-40-19-843/rule-output/ProfilerReport-1621197619/profiler-output/profiler-reports/OverallFrameworkMetrics.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/xgboost-2021-05-16-20-40-19-843/rule-output/ProfilerRepor